### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [162]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [163]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [164]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [165]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[100]

"1.  Software publishing SuperBase 4 windows v.1.3           --->$80\n\n2.  OCR System ReadRight v.3.1 for Windows                  --->$65\n\n3.  OCR System ReadRight  v.2.01 for DOS                    --->$65\n\n4.  Unregistered Zortech 32 bit C++ Compiler v.3.1          --->$ 250\n     with Multiscope windows Debugger,\n     WhiteWater Resource Toolkit, Library Source Code\n\n5.  Glockenspiel/ImageSoft Commonview 2 Windows\n     Applications Framework for Borland C++                 --->$70\n\n6.  Spontaneous Assembly Library With Source Code           --->$50\n\n7.  Microsoft Macro Assembly 6.0                            --->$50\n\n8.  Microsoft Windows v.3.1 SDK Documentation               --->$125\n\n9.  Microsoft FoxPro V.2.0                                  --->$75\n\n10.  WordPerfect 5.0 Developer's Toolkit                    --->$20\n\n11.  Kedwell Software DataBoss v.3.5 C Code Generator       --->$100\n\n12.  Kedwell InstallBoss v.2.0 Installation Generator       --->$35\n\

In [166]:
newsgroups_train.target[100]

6

In [167]:
newsgroups_test.target_names[6]

'misc.forsale'

In [168]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [169]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [170]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [171]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [172]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [173]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [174]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [175]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [176]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [177]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [178]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [179]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [180]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [181]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [182]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [183]:
newsgroups_test.data[0]

'I am a little confused on all of the models of the 88-89 bonnevilles.\nI have heard of the LE SE LSE SSE SSEI. Could someone tell me the\ndifferences are far as features or performance. I am also curious to\nknow what the book value is for prefereably the 89 model. And how much\nless than book value can you usually get them for. In other words how\nmuch are they in demand this time of year. I have heard that the mid-spring\nearly summer is the best time to buy.'

In [184]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [185]:
#Tomamos 5 documentos al azar
random_idx=[105,6,7065,43,7056]

for i in random_idx:
  print(f"Nuevo valor de index: {i}")
  print(newsgroups_test.target_names[newsgroups_train.target[i]])
  #newsgroups_test.target_names

  print("--------------------")
  print("Obtengo los 5 mayores valores de similaridad coseno, es decir los textos que tienen mas similaridad")
  cos=cosine_similarity(X_train[i], X_train)[0]
  max5=np.partition(cos, -5)
  print(max5)

  print("--------------------")
  print("Obtengo los indices")
  ind = np.argpartition(cos,-5)[-5:]
  print(ind)

  print("--------------------")
  for i in ind:
    # print(X_train.data[i])
    print(f"Imprimo la categoria del elemento {i}")
    print(newsgroups_test.target_names[newsgroups_train.target[i]])

  print("------------------------------------------------")



Nuevo valor de index: 105
sci.electronics
--------------------
Obtengo los 5 mayores valores de similaridad coseno, es decir los textos que tienen mas similaridad
[0.02533324 0.0158802  0.03572111 ... 0.39999896 0.44872036 1.        ]
--------------------
Obtengo los indices
[ 4248 10757  6664 10220   105]
--------------------
Imprimo la categoria del elemento 4248
sci.electronics
Imprimo la categoria del elemento 10757
misc.forsale
Imprimo la categoria del elemento 6664
sci.electronics
Imprimo la categoria del elemento 10220
sci.electronics
Imprimo la categoria del elemento 105
sci.electronics
------------------------------------------------
Nuevo valor de index: 6
sci.med
--------------------
Obtengo los 5 mayores valores de similaridad coseno, es decir los textos que tienen mas similaridad
[0.01973671 0.01160797 0.00648172 ... 1.         0.15267497 0.1495182 ]
--------------------
Obtengo los indices
[9623 8119    6 4847  804]
--------------------
Imprimo la categoria del elemento 9

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [186]:
#Vamos a tratar de maximizar el f1_score utilizando ComplementNB

c_nb = ComplementNB()
c_nb.fit(X_train, y_train)

#Entrenamos
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  c_nb.predict(X_test)

#Vemos el f1 score
f1_score(y_test, y_pred, average='macro')

#Vemos que mejoró

0.692953349950875

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


In [187]:
#Tomo una palabra al azar
random_idx=25780 #caramelizing

print("Probamos para una palabra al azar: ")
print(idx2word[random_idx])

#Transpongo la matrix documento-termino
X_T=X_train.T

print("-------------------------------")
print("Obtenemos los 5 mayores")
cos=cosine_similarity(X_T[random_idx], X_T)[0]
max5=np.partition(cos, -5)
print(max5)

print("-------------------------------")
print("Obtengo los indices")
ind = np.argpartition(cos,-5)[-5:]
print(ind)

for i in ind:
  print("-----------")
  print(f"Imprimo la categoria del elemento {i}")
  print(idx2word[i])

Probamos para una palabra al azar: 
caramelizing
-------------------------------
Obtenemos los 5 mayores
[0. 0. 0. ... 1. 1. 1.]
-------------------------------
Obtengo los indices
[66065 22429 25779 35142 72751]
-----------
Imprimo la categoria del elemento 66065
nitrosamines
-----------
Imprimo la categoria del elemento 22429
bergamot
-----------
Imprimo la categoria del elemento 25779
caramel
-----------
Imprimo la categoria del elemento 35142
drippy
-----------
Imprimo la categoria del elemento 72751
prawns
